In [ ]:
!pip install autogluon.tabular[all]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.8 

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

import numpy as np

train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 1000
train_data = train_data.sample(n=subsample_size, random_state=0)
print(train_data.head())

label = 'occupation'
print("Summary of occupation column: \n", train_data['occupation'].describe())

test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]
test_data_nolabel = test_data.drop(columns=[label])

metric = 'accuracy'

       age workclass  fnlwgt      education  education-num  \
6118    51   Private   39264   Some-college             10   
23204   58   Private   51662           10th              6   
29590   40   Private  326310   Some-college             10   
18116   37   Private  222450        HS-grad              9   
33964   62   Private  109190      Bachelors             13   

            marital-status        occupation    relationship    race      sex  \
6118    Married-civ-spouse   Exec-managerial            Wife   White   Female   
23204   Married-civ-spouse     Other-service            Wife   White   Female   
29590   Married-civ-spouse      Craft-repair         Husband   White     Male   
18116        Never-married             Sales   Not-in-family   White     Male   
33964   Married-civ-spouse   Exec-managerial         Husband   White     Male   

       capital-gain  capital-loss  hours-per-week  native-country   class  
6118              0             0              40   United-State

In [ ]:
from autogluon.common import space

nn_options = {
    'num_epochs': 10,
    'learning_rate': space.Real(1e-4, 1e-2, default=5e-4, log=True),
    'activation': space.Categorical('relu', 'softrelu', 'tanh'),
    'dropout_prob': space.Real(0.0, 0.5, default=0.1),
}

gbm_options = {
    'num_boost_round': 100,
    'num_leaves': space.Int(lower=26, upper=66, default=36),
}

hyperparameters = {
                   'GBM': gbm_options,
                   'NN_TORCH': nn_options,
                  }

time_limit = 2*60
num_trials = 5
search_strategy = 'auto'

hyperparameter_tune_kwargs = {
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}

predictor = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data,
    time_limit=time_limit,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241002_224642"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.25 GB / 12.67 GB (88.7%)
Disk Space Avail:   180.49 GB / 225.83 GB (79.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
Fitted model: LightGBM/T1 ...
	0.37	 = Validation score   (accuracy)
	2.17s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	0.355	 = Validation score   (accuracy)
	1.02s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	0.375	 = Validation score   (accuracy)
	0.67s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	0.36	 = Validation score   (accuracy)
	0.79s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	0.375	 = Validation score   (accuracy)
	0.63s	 = Training   runtime
	0.01s	 = Validation runtime
Hyperparameter tuning model: NeuralNetTorch

+---------------------------------------------------+
| Configuration for experiment     NeuralNetTorch   |
+---------------------------------------------------+
| Search algorithm                 SearchGenerator  |
| Scheduler                        FIFOScheduler    |
| Number of trials                 5                |
+---------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20241002_224642/models/NeuralNetTorch


Fitted model: NeuralNetTorch/7292990d ...
	0.355	 = Validation score   (accuracy)
	19.23s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: NeuralNetTorch/eda90fa8 ...
	0.38	 = Validation score   (accuracy)
	26.73s	 = Training   runtime
	0.04s	 = Validation runtime
Fitted model: NeuralNetTorch/541f192f ...
	0.265	 = Validation score   (accuracy)
	9.1s	 = Training   runtime
	0.03s	 = Validation runtime


Fitting model: WeightedEnsemble_L2 ... Training model for up to 119.77s of the 52.04s of remaining time.
	Ensemble Weights: {'NeuralNetTorch/eda90fa8': 0.667, 'LightGBM/T3': 0.333}
	0.385	 = Validation score   (accuracy)
	0.23s	 = Training   runtime
	0.01s	 = Validation runtime
AutoGluon training complete, total runtime = 68.27s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 3506.9 rows/s (200 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20241002_224642")


In [ ]:
y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  ", list(y_pred)[:5])
perf = predictor.evaluate(test_data, auxiliary_metrics=False)

Predictions:   [' Adm-clerical', ' Craft-repair', ' Craft-repair', ' Sales', ' Other-service']


In [ ]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L2      0.385    accuracy       0.057030  27.640082                0.007681           0.233146            2       True          9
1  NeuralNetTorch/eda90fa8      0.380    accuracy       0.035732  26.732273                0.035732          26.732273            1       True          7
2              LightGBM/T5      0.375    accuracy       0.013097   0.633858                0.013097           0.633858            1       True          5
3              LightGBM/T3      0.375    accuracy       0.013616   0.674662                0.013616           0.674662            1       True          3
4              LightGBM/T1      0.370    accuracy       0.008218   2.167400                0.008218           2.167400            1       True          1
5             

In [ ]:
label = 'class'
test_data_nolabel = test_data.drop(columns=[label])
y_test = test_data[label]
save_path = 'agModels-predictClass'

predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data,
    num_bag_folds=5, num_bag_sets=1, num_stack_levels=1,
    hyperparameters = {'NN_TORCH': {'num_epochs': 2}, 'GBM': {'num_boost_round': 20}},
)

No path specified. Models will be saved in: "AutogluonModels/ag-20241002_224902"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.30 GB / 12.67 GB (81.3%)
Disk Space Avail:   180.46 GB / 225.83 GB (79.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

In [ ]:

predictor = TabularPredictor(label=label, eval_metric='f1', path=save_path).fit(
    train_data, auto_stack=True,
    time_limit=30, hyperparameters={'FASTAI': {'num_epochs': 10}, 'GBM': {'num_boost_round': 200}}
)
predictor.leaderboard(test_data)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.45 GB / 12.67 GB (82.5%)
Disk Space Avail:   180.46 GB / 225.83 GB (79.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Stack configuration (auto_stac

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.629437,0.68559,f1,1.131676,0.101076,36.134240,1.131676,0.101076,36.134240,1,True,1
1,WeightedEnsemble_L2,0.629437,0.68559,f1,1.134707,0.105160,36.143734,0.003031,0.004085,0.009494,2,True,2


In [ ]:
print(f'Prior to calibration (predictor.decision_threshold={predictor.decision_threshold}):')
scores = predictor.evaluate(test_data)

calibrated_decision_threshold = predictor.calibrate_decision_threshold()
predictor.set_decision_threshold(calibrated_decision_threshold)

print(f'After calibration (predictor.decision_threshold={predictor.decision_threshold}):')
scores_calibrated = predictor.evaluate(test_data)

Prior to calibration (predictor.decision_threshold=0.5):


Calibrating decision threshold to optimize metric f1 | Checking 51 thresholds...
Calibrating decision threshold via fine-grained search | Checking 38 thresholds...
	Base Threshold: 0.500	| val: 0.6856
	Best Threshold: 0.500	| val: 0.6856


After calibration (predictor.decision_threshold=0.5):


In [ ]:
for metric_name in scores:
    metric_score = scores[metric_name]
    metric_score_calibrated = scores_calibrated[metric_name]
    decision_threshold = predictor.decision_threshold
    print(f'decision_threshold={decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')

decision_threshold=0.500	| metric="f1"
	test_score uncalibrated: 0.6294
	test_score   calibrated: 0.6294
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="accuracy"
	test_score uncalibrated: 0.8472
	test_score   calibrated: 0.8472
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="balanced_accuracy"
	test_score uncalibrated: 0.7438
	test_score   calibrated: 0.7438
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="mcc"
	test_score uncalibrated: 0.5457
	test_score   calibrated: 0.5457
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="roc_auc"
	test_score uncalibrated: 0.8990
	test_score   calibrated: 0.8990
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="precision"
	test_score uncalibrated: 0.7411
	test_score   calibrated: 0.7411
	test_score        delta: 0.0000
decision_threshold=0.500	| metric="recall"
	test_score uncalibrated: 0.5470
	test_score   calibrated: 0.5470
	test_score        delta: 0.0

In [ ]:
predictor.set_decision_threshold(0.5)  # Reset decision threshold
for metric_name in ['f1', 'balanced_accuracy', 'mcc']:
    metric_score = predictor.evaluate(test_data, silent=True)[metric_name]
    calibrated_decision_threshold = predictor.calibrate_decision_threshold(metric=metric_name, verbose=False)
    metric_score_calibrated = predictor.evaluate(
        test_data, decision_threshold=calibrated_decision_threshold, silent=True
    )[metric_name]
    print(f'decision_threshold={calibrated_decision_threshold:.3f}\t| metric="{metric_name}"'
          f'\n\ttest_score uncalibrated: {metric_score:.4f}'
          f'\n\ttest_score   calibrated: {metric_score_calibrated:.4f}'
          f'\n\ttest_score        delta: {metric_score_calibrated-metric_score:.4f}')

decision_threshold=0.500	| metric="f1"
	test_score uncalibrated: 0.6294
	test_score   calibrated: 0.6294
	test_score        delta: 0.0000
decision_threshold=0.250	| metric="balanced_accuracy"
	test_score uncalibrated: 0.7438
	test_score   calibrated: 0.8120
	test_score        delta: 0.0682
decision_threshold=0.500	| metric="mcc"
	test_score uncalibrated: 0.5457
	test_score   calibrated: 0.5457
	test_score        delta: 0.0000


In [ ]:
predictor = TabularPredictor.load(save_path)  # `predictor.path` is another way to get the relative path needed to later load predictor.

In [ ]:
predictor.features()

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']

In [ ]:
datapoint = test_data_nolabel.iloc[[0]]  # Note: .iloc[0] won't work because it returns pandas Series instead of DataFrame
print(datapoint)
predictor.predict(datapoint)

   age workclass  fnlwgt education  education-num       marital-status  \
0   31   Private  169085      11th              7   Married-civ-spouse   

  occupation relationship    race      sex  capital-gain  capital-loss  \
0      Sales         Wife   White   Female             0             0   

   hours-per-week  native-country  
0              20   United-States  


,class
0,<=50K


In [ ]:
predictor.predict_proba(datapoint)  # returns a DataFrame that shows which probability corresponds to which class

,<=50K,>50K
0,0.951059,0.048941


In [ ]:
predictor.model_best

'WeightedEnsemble_L2'

In [ ]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.629437,0.68559,f1,0.907643,0.101076,36.134240,0.907643,0.101076,36.134240,1,True,1
1,WeightedEnsemble_L2,0.629437,0.68559,f1,0.910443,0.105160,36.143734,0.002800,0.004085,0.009494,2,True,2


In [ ]:
predictor.leaderboard(extra_info=True)

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,LightGBM_BAG_L1,0.68559,f1,0.101076,36.134240,0.101076,36.134240,1,True,1,...,"{'use_orig_features': True, 'max_base_models':...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...","[race, relationship, workclass, marital-status...",None,"{'learning_rate': 0.05, 'num_boost_round': 200}",{'num_boost_round': 83},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[],[WeightedEnsemble_L2]
1,WeightedEnsemble_L2,0.68559,f1,0.105160,36.143734,0.004085,0.009494,2,True,2,...,"{'use_orig_features': False, 'max_base_models'...",{},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[LightGBM_BAG_L1],None,"{'ensemble_size': 25, 'subsample_size': 1000000}",{'ensemble_size': 1},"{'max_memory_usage_ratio': 1.0, 'max_time_limi...",[LightGBM_BAG_L1],[]


In [ ]:
predictor.leaderboard(test_data, extra_metrics=['accuracy', 'balanced_accuracy', 'log_loss'])

,model,score_test,accuracy,balanced_accuracy,log_loss,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.629437,0.84717,0.743784,-0.334022,0.68559,f1,0.737538,0.101076,36.134240,0.737538,0.101076,36.134240,1,True,1
1,WeightedEnsemble_L2,0.629437,0.84717,0.743784,-0.334022,0.68559,f1,0.739656,0.105160,36.143734,0.002119,0.004085,0.009494,2,True,2


In [ ]:
i = 0
model_to_use = predictor.model_names()[i]
model_pred = predictor.predict(datapoint, model=model_to_use)
print("Prediction from %s model: %s" % (model_to_use, model_pred.iloc[0]))

Prediction from LightGBM_BAG_L1 model:  <=50K


In [ ]:
all_models = predictor.model_names()
model_to_use = all_models[i]
specific_model = predictor._trainer.load_model(model_to_use)


model_info = specific_model.get_info()
predictor_information = predictor.info()

In [ ]:
y_pred_proba = predictor.predict_proba(test_data_nolabel)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred_proba)

In [ ]:
perf = predictor.evaluate(test_data)

In [ ]:
predictor.feature_importance(test_data)

Computing feature importance via permutation shuffling for 14 features using 5000 rows with 5 shuffle sets...
	31.77s	= Expected runtime (6.35s per shuffle set)
	32.64s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
marital-status,0.117047,0.009311,4.765349e-06,5,0.136219,0.097875
capital-gain,0.083102,0.004023,6.569424e-07,5,0.091385,0.074819
education-num,0.071497,0.006756,9.453208e-06,5,0.085407,0.057587
age,0.055806,0.009717,1.059856e-04,5,0.075814,0.035799
occupation,0.055400,0.013509,3.925863e-04,5,0.083214,0.027585
relationship,0.020842,0.006809,1.192225e-03,5,0.034861,0.006822
hours-per-week,0.018326,0.006083,1.264850e-03,5,0.030850,0.005802
capital-loss,0.003712,0.002108,8.495976e-03,5,0.008052,-0.000628
education,0.000719,0.001339,1.480534e-01,5,0.003477,-0.002038
native-country,0.000000,0.000000,5.000000e-01,5,0.000000,0.000000


In [ ]:
predictor.persist()

num_test = 20
preds = np.array(['']*num_test, dtype='object')
for i in range(num_test):
    datapoint = test_data_nolabel.iloc[[i]]
    pred_numpy = predictor.predict(datapoint, as_pandas=False)
    preds[i] = pred_numpy[0]

perf = predictor.evaluate_predictions(y_test[:num_test], preds, auxiliary_metrics=True)
print("Predictions: ", preds)

predictor.unpersist()

Persisting 2 models in memory. Models will require 0.02% of memory.
Unpersisted 2 models: ['LightGBM_BAG_L1', 'WeightedEnsemble_L2']


Predictions:  [' <=50K' ' <=50K' ' >50K' ' <=50K' ' <=50K' ' >50K' ' >50K' ' >50K'
 ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K' ' <=50K'
 ' <=50K' ' >50K' ' >50K' ' <=50K']


['LightGBM_BAG_L1', 'WeightedEnsemble_L2']

In [ ]:
infer_limit = 0.00005
infer_limit_batch_size = 10000

predictor_infer_limit = TabularPredictor(label=label, eval_metric=metric).fit(
    train_data=train_data,
    time_limit=30,
    infer_limit=infer_limit,
    infer_limit_batch_size=infer_limit_batch_size,
)


predictor_infer_limit.persist()

predictor_infer_limit.leaderboard()

No path specified. Models will be saved in: "AutogluonModels/ag-20241002_225420"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.38 GB / 12.67 GB (81.9%)
Disk Space Avail:   180.45 GB / 225.83 GB (79.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.860,accuracy,0.006384,4.163042,0.006384,4.163042,1,True,7
1,WeightedEnsemble_L2,0.860,accuracy,0.007303,4.229732,0.000919,0.066690,2,True,14
2,NeuralNetTorch,0.850,accuracy,0.017671,5.051608,0.017671,5.051608,1,True,12
3,LightGBMXT,0.850,accuracy,0.022087,1.376632,0.022087,1.376632,1,True,3
4,XGBoost,0.845,accuracy,0.015198,0.676064,0.015198,0.676064,1,True,11
5,LightGBM,0.840,accuracy,0.014304,1.026128,0.014304,1.026128,1,True,4
6,NeuralNetFastAI,0.840,accuracy,0.019490,2.425289,0.019490,2.425289,1,True,10
7,RandomForestGini,0.840,accuracy,0.081948,1.714623,0.081948,1.714623,1,True,5
8,RandomForestEntr,0.835,accuracy,0.079388,1.531362,0.079388,1.531362,1,True,6
9,ExtraTreesEntr,0.820,accuracy,0.079293,0.930518,0.079293,0.930518,1,True,9


In [ ]:
test_data_batch = test_data.sample(infer_limit_batch_size, replace=True, ignore_index=True)

import time
time_start = time.time()
predictor_infer_limit.predict(test_data_batch)
time_end = time.time()

infer_time_per_row = (time_end - time_start) / len(test_data_batch)
rows_per_second = 1 / infer_time_per_row
infer_time_per_row_ratio = infer_time_per_row / infer_limit
is_constraint_satisfied = infer_time_per_row_ratio <= 1

print(f'Model is able to predict {round(rows_per_second, 1)} rows per second. (User-specified Throughput = {1 / infer_limit})')
print(f'Model uses {round(infer_time_per_row_ratio * 100, 1)}% of infer_limit time per row.')
print(f'Model satisfies inference constraint: {is_constraint_satisfied}')

Model is able to predict 178675.7 rows per second. (User-specified Throughput = 20000.0)
Model uses 11.2% of infer_limit time per row.
Model satisfies inference constraint: True


In [ ]:
additional_ensembles = predictor.fit_weighted_ensemble(expand_pareto_frontier=True)
print("Alternative ensembles you can use for prediction:", additional_ensembles)

predictor.leaderboard(only_pareto_frontier=True)

Fitting model: WeightedEnsemble_L2Best ...
	Ensemble Weights: {'LightGBM_BAG_L1': 1.0}
	0.6856	 = Validation score   (f1)
	0.01s	 = Training   runtime
	0.01s	 = Validation runtime


Alternative ensembles you can use for prediction: ['WeightedEnsemble_L2Best']


,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.68559,f1,0.101076,36.13424,0.101076,36.13424,1,True,1


In [ ]:
model_for_prediction = additional_ensembles[0]
predictions = predictor.predict(test_data, model=model_for_prediction)
predictor.delete_models(models_to_delete=additional_ensembles, dry_run=False)

Deleting model WeightedEnsemble_L2Best. All files under agModels-predictClass/models/WeightedEnsemble_L2Best will be removed.


In [ ]:
refit_model_map = predictor.refit_full()
print("Name of each refit-full model corresponding to a previous bagged ensemble:")
print(refit_model_map)
predictor.leaderboard(test_data)

Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models ...
Fitting model: LightGBM_BAG_L1_FULL ...
	0.48s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	Ensemble Weights: {'LightGBM_BAG_L1': 1.0}
	0.01s	 = Training   runtime
Updated best model to "WeightedEnsemble_L2_FULL" (Previously "WeightedEnsemble_L2"). AutoGluon will default to using "WeightedEnsemble_L2_FULL" for predict() and predict_proba().
Refit complete, total runtime = 0.59s ... Best model: "WeightedEnsemble_L2_FULL"


Name of each refit-full model corresponding to a previous bagged ensemble:
{'LightGBM_BAG_L1': 'LightGBM_BAG_L1_FULL', 'WeightedEnsemble_L2': 'WeightedEnsemble_L2_FULL'}


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1_FULL,0.634860,NaN,f1,0.119527,NaN,0.480683,0.119527,NaN,0.480683,1,True,3
1,WeightedEnsemble_L2_FULL,0.634860,NaN,f1,0.122809,NaN,0.490178,0.003281,NaN,0.009494,2,True,4
2,LightGBM_BAG_L1,0.629437,0.68559,f1,1.004111,0.101076,36.134240,1.004111,0.101076,36.134240,1,True,1
3,WeightedEnsemble_L2,0.629437,0.68559,f1,1.007080,0.105160,36.143734,0.002969,0.004085,0.009494,2,True,2


In [ ]:
student_models = predictor.distill(time_limit=30)
print(student_models)
preds_student = predictor.predict(test_data_nolabel, model=student_models[0])
print(f"predictions from {student_models[0]}:", list(preds_student)[:5])
predictor.leaderboard(test_data)

Distilling with teacher='WeightedEnsemble_L2_FULL', teacher_preds=soft, augment_method=spunge ...
SPUNGE: Augmenting training data with 4000 synthetic samples for distillation...
Distilling with each of these student models: ['LightGBM_DSTL', 'CatBoost_DSTL', 'RandomForestMSE_DSTL', 'NeuralNetTorch_DSTL']
Fitting 4 L1 models ...
Fitting model: LightGBM_DSTL ... Training model for up to 30.0s of the 30.0s of remaining time.
		pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: workclass: object, education: object, marital-status: object, occupation: object, relationship: object, race: object, native-country: object
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/autogluon/core/trainer/abstract_trainer.py", line 1904, in _train_and_save
    model = self._train_single(X, y, model, X_val, y_val, total_resources=total_resources, **model_fit_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/autogluon/core/train

['RandomForestMSE_DSTL', 'WeightedEnsemble_L2_DSTL']
predictions from RandomForestMSE_DSTL: [' <=50K', ' <=50K', ' >50K', ' <=50K', ' <=50K']


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1_FULL,0.634860,NaN,f1,0.097104,NaN,0.480683,0.097104,NaN,0.480683,1,True,3
1,WeightedEnsemble_L2_FULL,0.634860,NaN,f1,0.099077,NaN,0.490178,0.001974,NaN,0.009494,2,True,4
2,LightGBM_BAG_L1,0.629437,0.685590,f1,0.745209,0.101076,36.134240,0.745209,0.101076,36.134240,1,True,1
3,WeightedEnsemble_L2,0.629437,0.685590,f1,0.747813,0.105160,36.143734,0.002604,0.004085,0.009494,2,True,2
4,RandomForestMSE_DSTL,0.607631,0.629213,mean_squared_error,0.568835,0.127872,8.042171,0.568835,0.127872,8.042171,1,True,5
5,WeightedEnsemble_L2_DSTL,0.607631,0.629213,mean_squared_error,0.576542,0.128920,8.047109,0.007707,0.001048,0.004937,2,True,6


In [ ]:
presets = ['good_quality', 'optimize_for_deployment']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets=presets, time_limit=60)

No path specified. Models will be saved in: "AutogluonModels/ag-20241002_225734"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.19 GB / 12.67 GB (80.4%)
Disk Space Avail:   180.32 GB / 225.83 GB (79.8%)
Presets specified: ['good_quality', 'optimize_for_deployment']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag

In [ ]:
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, hyperparameters='very_light', time_limit=30)

No path specified. Models will be saved in: "AutogluonModels/ag-20241002_225902"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.21 GB / 12.67 GB (80.5%)
Disk Space Avail:   180.32 GB / 225.83 GB (79.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

In [ ]:
excluded_model_types = ['KNN', 'NN_TORCH']
predictor_light = TabularPredictor(label=label, eval_metric=metric).fit(train_data, excluded_model_types=excluded_model_types, time_limit=30)

No path specified. Models will be saved in: "AutogluonModels/ag-20241002_225920"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.20 GB / 12.67 GB (80.5%)
Disk Space Avail:   180.31 GB / 225.83 GB (79.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : 

In [ ]:
test_data_preprocessed = predictor.transform_features(test_data)


predictions = predictor.predict(test_data_preprocessed, transform_features=False)